<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/solucion4_matching_catalogos_bse_autodata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Matching Optimizado v4: BSE vs Autodata

## ¿Qué hace este notebook?

Resuelve el problema de **vincular automáticamente dos catálogos de vehículos** que describen los mismos modelos con nomenclaturas distintas: el catálogo **BSE** (aprox. 33k registros) y el catálogo **Autodata** (aprox. 20k registros). Para cada registro de Autodata se encuentra el mejor equivalente en BSE, junto con un score de confianza [0, 1].

### Proceso general

1. **Preprocesamiento**: Ambos catálogos se normalizan para eliminar inconsistencias tipográficas, caracteres corruptos y variaciones de escritura. Se aplica un diccionario de sinónimos del dominio automotor (ej: `AA = A/A = aire acondicionado`, `TDI = TD Intercooler`, `5p = 5 puertas`, etc.) para que términos equivalentes queden representados con el mismo token antes de comparar. Los años erróneos del catálogo BSE (ej: `3020 → 2020`) también se corrigen en esta etapa.

2. **Índices de búsqueda**: Se construyen tres índices sobre BSE para acelerar el matching: un índice por marca+año (filtrado rápido), embeddings vectoriales con el modelo BGE-M3 (similitud semántica) e índices BM25 por marca (similitud léxica).

3. **Matching por registro**: Para cada registro Autodata se ejecuta un pipeline de tres fases:
   - **Pre-filtrado**: se limita el espacio de búsqueda a registros BSE de la misma marca y año cercano (±2 → ±5 → toda la marca, en fallback progresivo).
   - **Búsqueda híbrida**: se combinan similitud semántica (BGE-M3) y léxica (BM25) para obtener los 50 candidatos más prometedores.
   - **Reranking**: un cross-encoder (BGE-Reranker-v2-m3) decide el orden final. El score resultante se convierte a [0,1] con una función sigmoid de temperatura calibrada automáticamente, y se penaliza según la distancia temporal entre el año del registro Autodata y el rango de años del candidato BSE.

4. **Salida**: Para cada registro Autodata se reportan los 3 mejores candidatos BSE con su score, marca, modelo, rango de años, combustible y tipo. El resultado se exporta a CSV en dos formatos: completo (top-3) y resumido (mejor match solamente).


## 1️⃣ Instalación de dependencias

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Instalar librerías necesarias
!pip install -q sentence-transformers pandas numpy unidecode tqdm rank_bm25

## 2️⃣ Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import re
from tqdm.auto import tqdm
import warnings
import torch
from collections import defaultdict

warnings.filterwarnings('ignore')
print("Librerías importadas correctamente")
print(f"GPU disponible: {torch.cuda.is_available()}")

## 3️⃣ Cargar los archivos CSV

In [ ]:
# Cargar los datos (datatype str para evitar problemas de lectura)
df_bse = pd.read_csv(
    '/content/sample_data/CART_MATRICERO_BSE_202602190109.csv',
    dtype=str,
    keep_default_na=False
 )
df_autodata = pd.read_csv(
    '/content/sample_data/CART_PAD_AUTODATA_202602190107.csv',
    dtype=str,
    keep_default_na=False
 )

print(f"Registros BSE: {len(df_bse):,}")
print(f"Registros Autodata: {len(df_autodata):,}")
print("\nPrimeras filas de BSE:")
display(df_bse.head(10))
print("\nPrimeras filas de Autodata:")
display(df_autodata.head(10))

## 4️⃣ Funciones de Preprocesamiento

In [ ]:

# Detecta caracteres fuera del rango ASCII estándar para anticipar problemas de encoding
def detectar_caracteres_especiales(df, columna):
    todos_los_textos = " ".join(df[columna].astype(str).unique())
    caracteres_raros = set(re.findall(r'[^a-zA-Z0-9\s.,\-\/]', todos_los_textos))
    return caracteres_raros

print("Caracteres extraños en BSE:", detectar_caracteres_especiales(df_bse, 'MODELO_ORIGINAL'))
print("Caracteres extraños en Autodata:", detectar_caracteres_especiales(df_autodata, 'MODELO_A_ORIGINAL'))

def estandarizar_terminos_tecnicos(texto):
    """Normaliza sinónimos técnicos a tokens estándar únicos."""
    if not texto: return ""
    # normalizar_texto aplica .lower() antes de invocar esta función
    texto_procesado = texto

    # c/ → "con" solo cuando lo que sigue es una letra, no un dígito.
    # Esto evita alterar códigos de modelo con barra como "C/200" o "C/220".
    texto_procesado = re.sub(r'\bc\s*/\s*(?=[a-z])', ' con ', texto_procesado)
    texto_procesado = re.sub(r'\bp\s*/\s*', ' para ', texto_procesado)

    mapeos = {
        # AA, A/A, Aire y C/Aire son variantes del mismo equipamiento.
        # "C/Aire" llega aquí ya transformado en "con aire" por la expansión de c/ anterior.
        r"\baire\s+acondicionado\b|\ba/a\b|\baa\b|\baire\b|\bcon aire\b": " aire_acondicionado ",
        r"\bclimatizador\b|\bclim\b|\bclimaut\b": " climatizador ",
        r"\bpas\b|\bpax\b|\bpasajeros\b": " pasajeros ",
        r"\bhb\b|\bhatchback\b": " hatchback ",
        r"\bsb\b|\bsportback\b": " sportback ",
        r"\be\.full\b|\bex\.full\b|\bextra full\b": " extra_full ",
        r"\bs\.full\b|\bsuper full\b": " super_full ",
        r"\bcue\b|\bcuero\b": " cuero ",
        # Tech, Techo y T.Cielo designan el mismo equipamiento según especificación del negocio
        r"\btech\b|\btecho\b|\bt\.cielo\b": " techo_solar ",
        r"\bmb\b|\bmultim\b|\bmultimedia\b": " multimedia ",
        r"\bay\.est\b|\bay\.estac\b|\ba\.estac\b|\bp\.assi\b|\bp\.assist\b|\bpark assist\b": " ayuda_estacionamiento ",
        r"\bllan\b|\bllantas\b": " llantas ",
        # turbo_diesel_intercooler debe ir antes de turbo_diesel para que \btd\b
        # no consuma el "td" dentro de "TD Intercooler" antes de que el patrón compuesto pueda matchearlo
        r"\btdi\b|\btd intercooler\b|\bt\.diesel intercooler\b|\bturbo d intercooler\b": " turbo_diesel_intercooler ",
        r"\btd\b|\bt\.diesel\b|\bturbo d\b": " turbo_diesel ",
        # \baut\. sin \b final: el word boundary falla cuando el punto va seguido de espacio
        # o fin de string, ya que tanto "." como " " son caracteres no-palabra y no existe
        # boundary entre ellos. Tiptronic y S-Tronic se incluyen porque son transmisiones
        # automáticas, aunque algunos registros no lo indiquen explícitamente con "AUT".
        r"\btiptronic\b|\bs-tronic\b|\baut\.|\bautomatico\b": " automatico ",
        r"\b3-5p\b|\b3-5 ptas\b|\b3-5 puertas\b": " 3_5_puertas ",
        r"\b4-5p\b|\b4-5 ptas\b|\b4-5 puertas\b": " 4_5_puertas ",
        r"(?<!-)(?:\b2p\b|\b2 ptas\b|\b2 puertas\b)": " 2_puertas ",
        r"(?<!-)(?:\b3p\b|\b3 ptas\b|\b3 puertas\b)": " 3_puertas ",
        r"(?<!-)(?:\b4p\b|\b4 ptas\b|\b4 puertas\b)": " 4_puertas ",
        r"(?<!-)(?:\b5p\b|\b5 ptas\b|\b5 puertas\b)": " 5_puertas ",
        r"\badaptado a rural\b|\badaptada a rural\b|\bad\.?\s*rural\b": " furgon_reformado ",
        r"\blgo\b|\blargo\b": " largo ",
    }
    for patron, reemplazo in mapeos.items():
        texto_procesado = re.sub(patron, reemplazo, texto_procesado)
    return texto_procesado

def normalizar_texto(texto):
    """Repara caracteres extraños, unifica sinónimos y limpia símbolos."""
    if pd.isna(texto):
        return ""

    texto = str(texto).lower()

    MAPA_REPARACION = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ñ": "n",
        "ã¡": "a",
        "ã©": "e",
        "ã³": "o",
        "ãº": "u",
        "ã±": "n",
        "ã": "i",
        "¿": "o",
        "±": "n",
        "ý": "i",
        "ß": "a",
        "ð": "o",
        "â": "a",
        "²": "2",
        "³": "3",
        "¬": "",
        "·": "",
        "°": "",
        "º": "",
        "*": "",
        "+": "",
        "|": "",
        ")": "",
        '"': "",
        "!": "",
        "]": "",
        "[": "",
        "(": "",
        "?": "",
        "=": "",
    }

    for error, correccion in MAPA_REPARACION.items():
        texto = texto.replace(error, correccion)

    texto = unidecode(texto)
    texto = estandarizar_terminos_tecnicos(texto)

    # Los tokens temporales protegen puntos decimales (ej: "2,0" → "2.0") y guiones
    # internos (ej: "4x4-full") para que no se pierdan en la limpieza posterior de
    # caracteres no alfanuméricos
    decimal_token = "_decimal_tok_"
    hyphen_token  = "_hyphen_tok_"

    texto = re.sub(r"(?<=\d)[\.,](?=\d)", decimal_token, texto)
    texto = re.sub(r'(?<=[a-z0-9])-(?=[a-z0-9])', hyphen_token, texto)
    texto = re.sub(r'(?<=[a-z0-9])/(?=[a-z0-9])', '_', texto)
    texto = re.sub(r'\s+-\s+', ' ', texto)
    texto = re.sub(r"[^a-z0-9_\s]", " ", texto)
    texto = texto.replace(hyphen_token, '-')
    texto = texto.replace(decimal_token, '.')
    return re.sub(r"\s+", " ", texto).strip()

# El catálogo BSE contiene años erróneos en dos rangos: mayores a 3000 (ej: 3020)
# y en el rango 2985-2999 (ej: 2997). Todos corresponden a años reales menos 1000.
# El umbral 2030 cubre ambos casos sin afectar años actuales válidos.
ANIO_MAXIMO_VALIDO = 2030

def corregir_anio_bse(anio):
    """Corrige años erróneos restándoles 1000 (ej: 3020 → 2020, 2997 → 1997)."""
    if anio is None: return None
    return anio - 1000 if anio > ANIO_MAXIMO_VALIDO else anio

def extraer_anio_inicio_fin(rango_anios):
    if pd.isna(rango_anios): return None, None
    rango_str = str(rango_anios).strip()
    match = re.findall(r'(\d{4})', rango_str)
    if len(match) >= 2:
        return corregir_anio_bse(int(match[0])), corregir_anio_bse(int(match[1]))
    elif len(match) == 1:
        anio = corregir_anio_bse(int(match[0]))
        return anio, anio
    return None, None

def normalizar_marca(texto):
    if pd.isna(texto): return ""
    texto = str(texto).lower()
    texto = re.sub(r'<.*?>', '', texto)     # Elimina sufijos de país como <BRA>, <COL>
    texto = unidecode(texto)                # Citroën -> Citroen
    return texto.strip()

# Tabla de equivalencias entre nombres de marca en Autodata y en BSE.
# Necesaria porque algunos fabricantes aparecen con nombres distintos en cada catálogo
# (ej: "kia motors" en Autodata → "kia" en BSE, "dongfeng" → dos variantes en BSE).
ALIAS_MARCAS = {
    "polarsun":         ["polar sun"],
    "routebuggy":       ["route buggy"],
    "zhong tong":       ["zhongtong"],
    "zxauto":           ["zx auto"],
    "tongbao":          ["tong bao"],
    "leapmotor":        ["leap motor"],
    "kia motors":       ["kia"],
    "byd auto":         ["byd"],
    "bjc":              ["bjc-beijing"],
    "gwm":              ["gwm - [great wall motor]"],
    "gac - trumpchi":   ["gac"],
    "lynk co":          ["lynk y co"],
    "porsche replica":  ["porsche"],
    "brilliance sunra": ["brilliance"],
    "shineray":         ["shinerai"],
    "fuqi":             ["fuqui"],
    "dfsk":             ["dfm - dfsk (dong feng)"],
    "chana":            ["chana-changan"],
    "changan":          ["chana-changan"],
    "geely riddara":    ["riddara (geely)"],
    "dongfeng":         ["dfm - dfsk (dong feng)", "forthing (dongfeng)"],
    "daewoo - fso":     ["daewoo", "fso"],
    "vaz 1111 / uaz":   ["vaz", "uaz"],
}

def resolver_marcas_bse(marca_auto_norm):
    """Resuelve una marca normalizada de Autodata a lista de marcas BSE equivalentes."""
    if marca_auto_norm in ALIAS_MARCAS:
        return ALIAS_MARCAS[marca_auto_norm]
    return [marca_auto_norm]

def validar_normalizacion_contextual(df_bse, df_autodata, n_muestra=50):
    """Genera validación rápida con tabla de ejemplos y muestra real para revisión manual."""
    ejemplos = [
        ("c/diesel 2,0",          "con diesel 2.0"),
        ("p/lancha x-line",       "para lancha x-line"),
        ("S-TRONIC 3.3-3.8",      "automatico 3.3-3.8"),
        ("USM-2450-74-ST",        "usm-2450-74-st"),
        ("FULL - EXTRAFULL",      "full extrafull"),
        ("DX/LX 4-5p",            "dx_lx 4_5_puertas"),
        ("Adaptado a rural",       "furgon_reformado"),
        ("ad. rural",              "furgon_reformado"),
        ("AA 1.6",                 "aire_acondicionado 1.6"),
        ("A/A 1.6",                "aire_acondicionado 1.6"),
        ("C/Aire AUT.",            "aire_acondicionado automatico"),
        ("SEDAN AUT. FULL",        "sedan automatico full"),
        ("TECH 5P",                "techo_solar 5_puertas"),
        ("TD Intercooler 4x4",     "turbo_diesel_intercooler 4x4"),
        ("Turbo D Intercooler",    "turbo_diesel_intercooler"),
        ("TDI 2.0",                "turbo_diesel_intercooler 2.0"),
        ("TD 1.9",                 "turbo_diesel 1.9"),
    ]

    df_ejemplos = pd.DataFrame(ejemplos, columns=['entrada', 'salida_esperada'])
    df_ejemplos['salida_obtenida'] = df_ejemplos['entrada'].apply(normalizar_texto)
    df_ejemplos['ok'] = df_ejemplos['salida_obtenida'] == df_ejemplos['salida_esperada']

    serie_real = pd.concat([
        df_bse['MODELO_ORIGINAL'].astype(str),
        df_autodata['MODELO_A_ORIGINAL'].astype(str)
    ], ignore_index=True).dropna().drop_duplicates()

    n = min(n_muestra, len(serie_real))
    df_revision = pd.DataFrame({'entrada_real': serie_real.sample(n=n, random_state=42)})
    df_revision['salida_normalizada'] = df_revision['entrada_real'].apply(normalizar_texto)

    return df_ejemplos, df_revision

print("Funciones de preprocesamiento definidas")
print(f"Alias de marcas configurados: {len(ALIAS_MARCAS)} reglas")


## 5️⃣ Preprocesamiento de datos

In [ ]:
print("Iniciando preprocesamiento de catálogos...")

# --- PROCESAR BSE ---
df_bse['marca_norm'] = df_bse['MARCA'].apply(normalizar_marca)
df_bse['modelo_norm'] = df_bse['MODELO_ORIGINAL'].apply(normalizar_texto)
df_bse[['anio_inicio', 'anio_fin']] = df_bse['RANGO_ANIOS'].apply(
    lambda x: pd.Series(extraer_anio_inicio_fin(x))
)
# Filtrar registros sin años procesables
df_bse = df_bse[df_bse['anio_inicio'].notna()].copy()
df_bse.reset_index(drop=True, inplace=True)  # IMPORTANTE: resetear índice para indexación consistente con embeddings

# --- PROCESAR AUTODATA ---
df_autodata['marca_norm'] = df_autodata['MARCA_A'].apply(normalizar_marca)
df_autodata['modelo_norm'] = df_autodata['MODELO_A_ORIGINAL'].apply(normalizar_texto)
# Año técnico para matching (derivado desde texto)
df_autodata['anio'] = pd.to_numeric(df_autodata['ANIO_A'], errors='coerce').fillna(9999).astype(int)

print("Preprocesamiento completado.")
print(f"BSE limpio: {len(df_bse):,} registros | Autodata: {len(df_autodata):,} registros.")

print("\nEjemplo de preprocesamiento:")
print("\nBSE:")
display(df_bse[['MARCA', 'marca_norm', 'MODELO_ORIGINAL', 'modelo_norm', 'RANGO_ANIOS', 'anio_inicio', 'anio_fin']].head(10))
print("\nAutodata:")
display(df_autodata[['MARCA_A', 'marca_norm', 'MODELO_A_ORIGINAL', 'modelo_norm', 'ANIO_A', 'anio']].head(10))

# Validación recomendada antes de ejecutar matching completo
df_validacion_ejemplos, df_revision_50 = validar_normalizacion_contextual(df_bse, df_autodata, n_muestra=50)
print("\nValidación de ejemplos controlados:")
display(df_validacion_ejemplos)
print("\nMuestra de 50 casos reales para revisión manual:")
display(df_revision_50)

## 6️⃣ Cargar Modelos y Construir Índices Optimizados

In [ ]:
# Detectar dispositivo para los modelos de IA (Transformers sí usarán GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando device para IA: {device}")

# ========== 1. MODELOS DE IA ==========
print("\n[1/5] Cargando modelos de embeddings y reranker...")
# BGE-M3 genera los vectores y Reranker decide el mejor candidato
model_bi = SentenceTransformer('BAAI/bge-m3', device=device)
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', device=device)
print("Modelos cargados")

# ========== 2. ÍNDICE POR MARCA + AÑO (Pre-filtrado inteligente) ==========
print("\n[2/5] Construyendo índice de marca-año para filtrado rápido...")
marca_anio_index = defaultdict(lambda: defaultdict(list))

for idx, row in df_bse.iterrows():
    marca = row['marca_norm']
    try:
        anio_ini = int(row['anio_inicio'])
        anio_fin = int(row['anio_fin'])
        for year in range(anio_ini, anio_fin + 1):
            marca_anio_index[marca][year].append(idx)
    except (ValueError, TypeError):
        continue

print(f"Índice construido: {len(marca_anio_index)} marcas")

# ========== 3. EMBEDDINGS BSE (Búsqueda vectorial) ==========
print("\n[3/5] Generando embeddings BGE-M3 para BSE...")
bse_textos = (df_bse['marca_norm'] + " " + df_bse['modelo_norm']).tolist()

bse_embeddings = model_bi.encode(
    bse_textos,
    convert_to_tensor=False,
    batch_size=512,
    show_progress_bar=True,
    normalize_embeddings=True
).astype('float32')

print(f"Embeddings generados: {bse_embeddings.shape[0]} vectores ({bse_embeddings.shape[1]}D)")

# ========== 4. ÍNDICES BM25 POR MARCA ==========
print("\n[4/5] Construyendo índices BM25 por marca...")
bm25_cache = {}
marca_to_indices = defaultdict(list)

for idx, row in df_bse.iterrows():
    marca_to_indices[row['marca_norm']].append(idx)

for marca, indices in tqdm(marca_to_indices.items(), desc="BM25 por marca"):
    corpus_marca = [str(df_bse.loc[i, 'modelo_norm']).split() for i in indices]
    bm25_cache[marca] = {
        'bm25': BM25Okapi(corpus_marca),
        'indices': indices
    }

print(f"{len(bm25_cache)} índices BM25 cacheados")

# ========== 5. PRE-ENCODING AUTODATA + CALIBRACIÓN SIGMOID ==========
print("\n[5/5] Pre-encoding de queries Autodata y calibración de temperatura...")

# 5a. Batch encoding de todos los textos Autodata
# Evita encodear 1 query a la vez dentro del loop de matching (cuello de botella en GPU)
autodata_textos = (df_autodata['marca_norm'] + " " + df_autodata['modelo_norm']).tolist()
autodata_embeddings = model_bi.encode(
    autodata_textos,
    convert_to_tensor=False,
    batch_size=512,
    show_progress_bar=True,
    normalize_embeddings=True
).astype('float32')
print(f"Autodata embeddings pre-calculados: {autodata_embeddings.shape[0]} vectores ({autodata_embeddings.shape[1]}D)")

# 5b. Calibración automática de temperatura del sigmoid
# Lógica: muestreamos pares (query Autodata → top-1 BM25 por marca) y observamos
# la distribución de logits del reranker. Luego fijamos T tal que:
#   sigmoid(p80_logit / T) ≈ 0.95  →  T = p80 / ln(19) ≈ p80 / 2.944
# Esto garantiza que el 80% de los pares "candidatos" caigan en zona útil (> 0.5).
CALIBRACION_N = 150
pares_calibracion = []

autodata_muestra = df_autodata[df_autodata['marca_norm'].apply(
    lambda m: any(mb in marca_to_indices for mb in resolver_marcas_bse(m))
)].sample(min(CALIBRACION_N * 2, len(df_autodata)), random_state=42).reset_index(drop=True)

for _, row in autodata_muestra.iterrows():
    if len(pares_calibracion) >= CALIBRACION_N:
        break
    marca_auto = row['marca_norm']
    modelo_auto = row['modelo_norm']
    anio_auto  = row['anio']
    query_text = f"{marca_auto} {modelo_auto}"
    for m in resolver_marcas_bse(marca_auto):
        if m in bm25_cache:
            bm25_scores = bm25_cache[m]['bm25'].get_scores(modelo_auto.split())
            best_local  = int(np.argmax(bm25_scores))
            best_idx    = bm25_cache[m]['indices'][best_local]
            bse_text = (
                f"{df_bse.loc[best_idx, 'marca_norm']} "
                f"{df_bse.loc[best_idx, 'modelo_norm']} "
                f"{int(df_bse.loc[best_idx, 'anio_inicio'])}-{int(df_bse.loc[best_idx, 'anio_fin'])}"
            )
            pares_calibracion.append([
                f"{query_text} {anio_auto if anio_auto != 9999 else ''}",
                bse_text
            ])
            break

if pares_calibracion:
    logits_cal = reranker.predict(pares_calibracion)
    p10 = float(np.percentile(logits_cal, 10))
    p50 = float(np.percentile(logits_cal, 50))
    p80 = float(np.percentile(logits_cal, 80))
    # Temperatura acotada a [0.3, 2.0] para evitar extremos
    SIGMOID_TEMPERATURA = round(max(0.3, min(p80 / 2.944, 2.0)), 3) if p80 > 0.1 else 1.0
    print(f"\nCalibración sobre {len(pares_calibracion)} pares:")
    print(f"  Logits reranker — p10: {p10:.2f} | p50: {p50:.2f} | p80: {p80:.2f}")
    print(f"  Temperatura sigmoid calibrada automáticamente: {SIGMOID_TEMPERATURA}")
else:
    SIGMOID_TEMPERATURA = 1.0
    print("Calibración no disponible. Temperatura por defecto: 1.0")

print("\n" + "="*70)
print("SISTEMA OPTIMIZADO LISTO")
print("="*70)

## 7️⃣ Función de Matching Optimizada

In [ ]:

def calcular_penalizacion_anio(anio_query, anio_ini_bse, anio_fin_bse):
    """
    Penalización gradual según distancia temporal entre el año de Autodata y el rango BSE.
    Cuanto mayor es la distancia, menor es el multiplicador aplicado al score final.
    Retorna un multiplicador en [0.25, 1.0].
    """
    if anio_query == 9999:   # Año desconocido en Autodata: penalización leve pero presente
        return 0.9

    if anio_ini_bse <= anio_query <= anio_fin_bse:
        return 1.0  # El año cae dentro del rango BSE: sin penalización

    distancia = (anio_ini_bse - anio_query) if anio_query < anio_ini_bse else (anio_query - anio_fin_bse)

    if distancia == 1:    return 0.95
    elif distancia == 2:  return 0.88
    elif distancia == 3:  return 0.78
    elif distancia <= 5:  return 0.65
    elif distancia <= 10: return 0.45
    else:                 return 0.25

def sigmoid_temperatura(x, temperatura=1.0):
    """Convierte logits del reranker a [0,1] con escala calibrada para comparabilidad entre registros."""
    return 1.0 / (1.0 + np.exp(-x / temperatura))

def encontrar_matches_optimizado(row_autodata, top_k=3, top_candidates=50, query_embedding=None):
    """
    Pipeline de matching en 3 fases:
    1. Pre-filtrado por marca + año con fallback progresivo (±2 → ±5 → toda la marca).
    2. Búsqueda híbrida: similitud semántica (BGE-M3) + léxica (BM25).
    3. Reranking con cross-encoder y penalización temporal aplicada sobre el score final.

    El score resultante (score_comparable) es globalmente consistente entre registros,
    lo que permite comparar la confianza de distintos matches entre sí.
    """
    marca_auto  = row_autodata['marca_norm']
    anio_auto   = row_autodata['anio']
    modelo_auto = row_autodata['modelo_norm']
    query_text  = f"{marca_auto} {modelo_auto}"

    # ========== FASE 1: PRE-FILTRADO CON FALLBACK PROGRESIVO ==========
    marcas_bse = resolver_marcas_bse(marca_auto)

    # Se usa set comprehension para deduplicar índices cuando una marca tiene múltiples
    # alias en BSE y varios de ellos apuntan al mismo registro
    indices_marca = list({
        idx
        for m in marcas_bse
        if m in marca_to_indices
        for idx in marca_to_indices[m]
    })

    if not indices_marca:
        return []

    indices_candidatos = []
    if anio_auto != 9999:
        # Fallback progresivo por radio de año:
        #   - ±2 años: conjunto más preciso y restrictivo
        #   - ±5 años: ampliación si ±2 devuelve menos de 10 candidatos
        # Se requieren al menos 10 candidatos para que la búsqueda híbrida y el reranker
        # tengan suficiente variedad para discriminar correctamente.
        # Si ningún radio alcanza ese mínimo, se usa el mejor resultado parcial encontrado
        # (aunque sea < 10) en lugar de descartarlo. Solo si ambos radios devuelven exactamente
        # 0 candidatos se cae al universo completo de la marca como último recurso.
        mejor_candidatos_parciales = set()
        for radio in [2, 5]:
            candidatos_set = set()
            for m in marcas_bse:
                if m in marca_anio_index:
                    for offset in range(-radio, radio + 1):
                        year = anio_auto + offset
                        if year in marca_anio_index[m]:
                            candidatos_set.update(marca_anio_index[m][year])
            if len(candidatos_set) >= 10:
                indices_candidatos = list(candidatos_set)
                break
            # Conservar el resultado parcial más amplio por si ningún radio alcanza el mínimo
            if len(candidatos_set) > len(mejor_candidatos_parciales):
                mejor_candidatos_parciales = candidatos_set

        if not indices_candidatos:
            if mejor_candidatos_parciales:
                indices_candidatos = list(mejor_candidatos_parciales)
            else:
                # Ningún año cercano encontró candidatos: usar todos los registros de la marca
                indices_candidatos = indices_marca
    else:
        # Año desconocido: no es posible filtrar por rango temporal
        indices_candidatos = indices_marca

    if not indices_candidatos:
        return []

    # ========== FASE 2: BÚSQUEDA HÍBRIDA ==========
    if query_embedding is not None:
        q_emb = query_embedding.reshape(1, -1).astype('float32')
    else:
        q_emb = model_bi.encode([query_text], normalize_embeddings=True).astype('float32')

    candidate_indices    = np.array(indices_candidatos)
    candidate_embeddings = bse_embeddings[candidate_indices]
    similarities         = np.dot(candidate_embeddings, q_emb.T).flatten()

    # Se toma el doble de top_candidates en semántica para que BM25 tenga suficiente pool
    # donde compensar casos donde el sentido semántico no es suficiente por sí solo
    top_sem_count     = min(top_candidates * 2, len(candidate_indices))
    top_sem_positions = np.argsort(similarities)[::-1][:top_sem_count]
    semantic_results  = [(int(candidate_indices[pos]), float(similarities[pos])) for pos in top_sem_positions]

    if not semantic_results:
        return []

    # BM25 léxico: complementa la búsqueda semántica para términos técnicos exactos
    # (versiones, cilindradas, abreviaturas) donde la similitud vectorial puede fallar
    bm25_dict       = {}
    tokenized_query = modelo_auto.split()
    for m in marcas_bse:
        if m in bm25_cache:
            bm25_data            = bm25_cache[m]
            bm25_model           = bm25_data['bm25']
            bm25_indices_locales = bm25_data['indices']
            bm25_scores          = bm25_model.get_scores(tokenized_query)
            if bm25_scores.max() > 0:
                bm25_scores = bm25_scores / bm25_scores.max()  # Normalización al rango [0, 1]
            for idx, score in zip(bm25_indices_locales, bm25_scores):
                if idx not in bm25_dict or score > bm25_dict[idx]:
                    bm25_dict[idx] = score

    # Combinación ponderada: más peso a semántica (70%) que a léxico (30%).
    # La penalización de año no se aplica aquí sino después del reranker,
    # para no distorsionar la selección de candidatos en esta fase intermedia.
    combined_scores = []
    for idx, sem_score in semantic_results:
        lex_score    = bm25_dict.get(idx, 0)
        hybrid_score = 0.70 * sem_score + 0.30 * lex_score
        combined_scores.append((idx, hybrid_score))

    combined_scores.sort(key=lambda x: x[1], reverse=True)
    top_hybrid = combined_scores[:top_candidates]

    if not top_hybrid:
        return []

    # ========== FASE 3: RERANKING + SCORE COMPARABLE ==========
    indices_finales   = [idx for idx, _ in top_hybrid]
    hybrid_scores_raw = np.array([score for _, score in top_hybrid], dtype=float)

    # El texto del par incluye el año de Autodata para que el reranker
    # pueda considerarlo al comparar contra el rango de años BSE
    pairs = [
        [
            f"{query_text} {anio_auto if anio_auto != 9999 else ''}",
            f"{df_bse.loc[idx, 'marca_norm']} {df_bse.loc[idx, 'modelo_norm']} "
            f"{int(df_bse.loc[idx, 'anio_inicio'])}-{int(df_bse.loc[idx, 'anio_fin'])}"
        ]
        for idx in indices_finales
    ]

    rerank_scores = reranker.predict(pairs)

    # score_comparable combina reranker (85%) y score híbrido (15%).
    # La sigmoid con temperatura calibrada convierte los logits del reranker a [0,1]
    # de forma consistente entre registros, lo que permite comparar scores de distintas queries.
    rerank_global    = sigmoid_temperatura(rerank_scores, temperatura=SIGMOID_TEMPERATURA)
    hybrid_global    = np.clip(hybrid_scores_raw, 0, 1)
    score_comparable = 0.85 * rerank_global + 0.15 * hybrid_global

    # La penalización temporal se aplica sobre el score_comparable final para que tenga
    # peso real en la decisión: un candidato temporalmente lejano queda efectivamente
    # penalizado aunque su similitud de texto sea alta
    year_penalties = np.array([
        calcular_penalizacion_anio(
            anio_auto,
            df_bse.loc[idx, 'anio_inicio'],
            df_bse.loc[idx, 'anio_fin']
        )
        for idx in indices_finales
    ])
    score_comparable = score_comparable * year_penalties

    # Ordenar por score_comparable garantiza que el top_k retornado sea coherente
    # y directamente comparable entre distintas queries
    top_k_indices = np.argsort(score_comparable)[::-1][:top_k]

    return [{
        'ID_AUTODATA_BSE':  df_bse.loc[indices_finales[i], 'ID_AUTODATA_BSE'],
        'score_comparable': round(float(score_comparable[i]), 4),
        'marca_bse':        df_bse.loc[indices_finales[i], 'MARCA'],
        'modelo_bse':       df_bse.loc[indices_finales[i], 'MODELO_ORIGINAL'],
        'rango_anios_bse':  df_bse.loc[indices_finales[i], 'RANGO_ANIOS'],
        'comb_bse':         df_bse.loc[indices_finales[i], 'COMB'],
        'tipo_bse':         df_bse.loc[indices_finales[i], 'TIPO'],
    } for i in top_k_indices]

print("Función de matching optimizada definida")


## 8️⃣ Ejecutar matching para todo el catálogo Autodata

In [ ]:

TOP_K = 3
resultados_finales = []

print(f"Iniciando matching de {len(df_autodata):,} registros...")
print("Estimado: ~30-40 minutos en T4 GPU (embeddings Autodata pre-calculados)\n")

for enum_idx, (_, row) in enumerate(tqdm(df_autodata.iterrows(), total=len(df_autodata), desc="Matching")):
    matches = encontrar_matches_optimizado(
        row,
        top_k=TOP_K,
        query_embedding=autodata_embeddings[enum_idx]
    )

    # Estructura base con valores vacíos para garantizar filas homogéneas en el DataFrame final,
    # incluso para registros sin ningún match encontrado
    res = {
        'ID_AUTADATA':         row['ID_AUTADATA'],
        'MARCA_A':             row['MARCA_A'],
        'MODELO_A_ORIGINAL':   row['MODELO_A_ORIGINAL'],
        'ANIO_A':              row['ANIO_A'],

        'SCORE_COMPARABLE':    0.0,
        'SCORE_1':             0.0,
        'SCORE_2':             0.0,
        'SCORE_3':             0.0,

        'ID_AUTODATA_BSE_1':   'NO_MATCH',
        'ID_AUTODATA_BSE_2':   '',
        'ID_AUTODATA_BSE_3':   '',
        'OBSERVACION':         '',

        'MARCA_BSE_1':         '',
        'MARCA_BSE_2':         '',
        'MARCA_BSE_3':         '',

        'MODELO_BSE_1':        '',
        'MODELO_BSE_2':        '',
        'MODELO_BSE_3':        '',

        'RANGO_ANIOS_BSE_1':   '',
        'RANGO_ANIOS_BSE_2':   '',
        'RANGO_ANIOS_BSE_3':   '',

        'COMB_BSE':            '',
        'TIPO_BSE':            ''
    }

    if matches:
        first_id = str(matches[0].get('ID_AUTODATA_BSE', '')).strip()
        if first_id in ('', '0'):
            res['OBSERVACION'] = 'MATCH_SIN_ID_BSE'

        # SCORE_COMPARABLE refleja el score del mejor candidato (posición 1)
        res['SCORE_COMPARABLE'] = matches[0].get('score_comparable', 0.0)

        res['COMB_BSE']         = matches[0].get('comb_bse', '')
        res['TIPO_BSE']         = matches[0].get('tipo_bse', '')

        for i, m in enumerate(matches[:TOP_K]):
            suffix = f"_{i+1}"
            # Todos los scores reportados son score_comparable: globalmente consistentes entre registros
            res[f'SCORE{suffix}']           = m['score_comparable']
            res[f'ID_AUTODATA_BSE{suffix}'] = m['ID_AUTODATA_BSE']
            res[f'MARCA_BSE{suffix}']       = m['marca_bse']
            res[f'MODELO_BSE{suffix}']      = m['modelo_bse']
            res[f'RANGO_ANIOS_BSE{suffix}'] = m['rango_anios_bse']
    resultados_finales.append(res)

df_resultados = pd.DataFrame(resultados_finales)

# 'NO_MATCH' en ID_AUTODATA_BSE_1 es el centinela explícito de registros sin match.
# Se evita usar SCORE_COMPARABLE == 0 porque matches reales de baja confianza también tienen score bajo.
n_sin_match = (df_resultados['ID_AUTODATA_BSE_1'] == 'NO_MATCH').sum()
n_con_match = (df_resultados['ID_AUTODATA_BSE_1'] != 'NO_MATCH').sum()

print("\n" + "="*70)
print("MATCHING COMPLETADO")
print("="*70)
print(f"Total procesado: {len(df_resultados):,} registros")
print(f"Sin matches:     {n_sin_match:,}")
print(f"Con matches:     {n_con_match:,}")

df_con_match = df_resultados[df_resultados['ID_AUTODATA_BSE_1'] != 'NO_MATCH']
if len(df_con_match) > 0:
    print(f"\nScore comparable promedio: {df_con_match['SCORE_COMPARABLE'].mean():.3f}")

display(df_resultados.head(10))


## 9️⃣ Análisis de Calidad y Exportación

In [ ]:

print("\nDISTRIBUCIÓN DE SCORE_COMPARABLE (GLOBAL):\n")

# Se excluyen registros sin match para calcular estadísticas solo sobre matches válidos.
# El criterio es ID_AUTODATA_BSE_1 != 'NO_MATCH' y no SCORE_COMPARABLE > 0,
# porque matches de baja confianza también tienen scores bajos pero son válidos.
df_con_match_global = df_resultados[df_resultados['ID_AUTODATA_BSE_1'] != 'NO_MATCH'].copy()

# Si no hay ningún match en el dataset, quantile() sobre una Serie vacía fallaría
if len(df_con_match_global) == 0:
    print("No se encontraron matches en el dataset. No hay estadísticas de score disponibles.")
    print(f"\nSin match (NO_MATCH): {len(df_resultados):,}")
else:
    p25 = df_con_match_global['SCORE_COMPARABLE'].quantile(0.25)
    p50 = df_con_match_global['SCORE_COMPARABLE'].quantile(0.50)
    p75 = df_con_match_global['SCORE_COMPARABLE'].quantile(0.75)

    print(f"Umbrales calculados sobre {len(df_con_match_global):,} registros con match:")
    print(f"  p25 = {p25:.4f} | p50 = {p50:.4f} | p75 = {p75:.4f}")
    print(f"  Temperatura sigmoid usada: {SIGMOID_TEMPERATURA}\n")

    print(f"Score >= {p75:.3f} (Alta confianza,    top 25%):     {(df_con_match_global['SCORE_COMPARABLE'] >= p75).sum():,}")
    print(f"Score {p50:.3f}-{p75:.3f} (Media confianza, 50-75%):  {((df_con_match_global['SCORE_COMPARABLE'] >= p50) & (df_con_match_global['SCORE_COMPARABLE'] < p75)).sum():,}")
    print(f"Score {p25:.3f}-{p50:.3f} (Revisión humana, 25-50%): {((df_con_match_global['SCORE_COMPARABLE'] >= p25) & (df_con_match_global['SCORE_COMPARABLE'] < p50)).sum():,}")
    print(f"Score <  {p25:.3f} (Baja confianza,   bot 25%):     {(df_con_match_global['SCORE_COMPARABLE'] < p25).sum():,}")
    print(f"\nSin match (NO_MATCH): {(df_resultados['ID_AUTODATA_BSE_1'] == 'NO_MATCH').sum():,}")

archivo_salida = 'resultados_matching_completo_v4.csv'
df_resultados.to_csv(archivo_salida, index=False, encoding='utf-8-sig')
print(f"\nResultados exportados a: {archivo_salida}")

print("\n" + "="*70)
print("========= PROCESO COMPLETADO EXITOSAMENTE =========")
print("="*70)


In [ ]:

columnas_origen = [
    'ID_AUTADATA',
    'ID_AUTODATA_BSE_1',
    'SCORE_COMPARABLE',
    'MARCA_A',
    'MARCA_BSE_1',
    'ANIO_A',
    'RANGO_ANIOS_BSE_1',
    'MODELO_A_ORIGINAL',
    'MODELO_BSE_1',
    'COMB_BSE',
    'TIPO_BSE',
    'OBSERVACION'
]

renombrar_columnas = {
    'ID_AUTADATA':       'ID_AUTODATA',
    'ID_AUTODATA_BSE_1': 'ID_AUTODATA_BSE',
    'MARCA_BSE_1':       'MARCA_BSE',
    'RANGO_ANIOS_BSE_1': 'RANGO_ANIOS_BSE',
    'MODELO_A_ORIGINAL': 'MODELO_A',
    'MODELO_BSE_1':      'MODELO_BSE'
}

df_filtrado = df_resultados[columnas_origen].rename(columns=renombrar_columnas).copy()
print("DataFrame 'df_filtrado' creado con columnas finales estandarizadas.")
display(df_filtrado.head(10))

archivo_filtrado_salida = "resultados_matching_resumido_v4.csv"
df_filtrado.to_csv(archivo_filtrado_salida, index=False, encoding="utf-8-sig")
print(f"DataFrame filtrado exportado a: {archivo_filtrado_salida}")


In [ ]:
df_bse.to_csv('debug_df_bse.csv', index=False, encoding='utf-8-sig')
df_autodata.to_csv('debug_df_autodata.csv', index=False, encoding='utf-8-sig')
df_validacion_ejemplos.to_csv('debug_df_validacion_ejemplos.csv', index=False, encoding='utf-8-sig')
df_revision_50.to_csv('debug_df_revision_50.csv', index=False, encoding='utf-8-sig')

print("4 archivos exportados.")